Kasus:

Membedakan review positif dan negatif dari pelanggan kepada suatu restoran.

Dataset yang digunakan: [Yelp](https://www.kaggle.com/marklvl/sentiment-labelled-sentences-data-set) (berisi review dari beberapa restoran di US)

Sumber belajar/explore NLP-Sentiment Analysis: [Dicoding-Pengembangan Machine Learning](https://www.dicoding.com/academies/185)

* Load dataset pada Colaboratory.
* Buat dataframe dari dataset agar lebih mudah diproses

In [4]:
import pandas as pd
df = pd.read_csv('yelp_labelled.txt', names=['sentence', 'label'], sep='\t')

* Melihat 5 sampel terakhir dari datase

In [5]:
df.tail()

,sentence,label
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0
999,"Then, as if I hadn't wasted enough of my life ...",0


* Bagi dataset menjadi train dan test set.

In [6]:
from sklearn.model_selection import train_test_split
kalimat = df['sentence'].values
y = df['label'].values
kalimat_latih, kalimat_test, y_latih, y_test = train_test_split(kalimat, y, test_size=0.2)

* Melakukan tokenisasi dengan fungsi tokenizer supaya teks  dapat dipahami oleh model. Menerapkan tokenisasi pada data latih dan data test.

  Proses mengonversi kata-kata ke dalam bilangan numerik dapat kita sebut juga sebagai **tokenisasi**. 

  Parameter **num_words** adalah jumlah kata yang akan dikonversi ke dalam token/bilangan numerik. Jika parameter num_words diatur sebanyak 15, hanya 15 kata yang paling sering muncul. 15 kata tersebut akan ditokenisasi dari seluruh kata pada dataset. 

  Sedangkan parameter **oov_token** adalah parameter yang berfungsi untuk mengganti kata-kata yang tidak ditokenisasi menjadi karakter tertentu.

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=250, oov_token='x')
tokenizer.fit_on_texts(kalimat_latih) 
tokenizer.fit_on_texts(kalimat_test)

* Mengubah text yang telah dibuat sebelumnya ke dalam bentuk sequence menggunakan fungsi text_to_sequences.

 Sebuah **sekuens** adalah sebuah larik yang berisi kumpulan token sesuai dengan setiap kata pada sebuah kalimat dalam teks.

In [8]:
sekuens_latih = tokenizer.texts_to_sequences(kalimat_latih)
sekuens_test = tokenizer.texts_to_sequences(kalimat_test)

* Menggunakan fungsi pad_sequences agar setiap sequence sama panjang

  Pada padding, setiap sequence dibuat sama panjang dengan menambahkan nilai 0 secara sufiks atau prefiks hingga mencapai panjang maksimum sequence. Selain itu padding juga dapat memotong sequence hingga panjangnya sesuai dengan panjang maksimum sequence.

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_latih = pad_sequences(sekuens_latih, maxlen=20) 
padded_test = pad_sequences(sekuens_test, maxlen=20)

* Untuk arsitektur yang akan digunakan adalah layer embedding. 

  Argumen pertama sesuai dengan jumlah vocabulary/kata yang kita pakai pada tokenizer. 
  
  Argumen selanjutnya adalah dimensi embedding, dan input_length yang merupakan panjang dari sequence. 
  
  Terapkan fungsi GlobalAveragePooling1D.

In [10]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(250, 16, input_length=20),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

* Panggil fungsi compile. 

  Karena ini merupakan kasus klasifikasi dua kelas, maka menggunakan 'binary_crossentropy' sebagai nilai dari loss.

  Menggunakan optimize 'adam'

  Untuk menghitung seberapa sering  hasil prediksi sesuai dengan label, maka menggunakan metrics=['accuracy'].

In [11]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

* Melakukan pelatihan model

In [12]:
num_epochs = 30
history = model.fit(padded_latih, y_latih, epochs=num_epochs, validation_data=(padded_test, y_test), verbose=2)

Epoch 1/30
25/25 - 1s - loss: 0.6930 - accuracy: 0.5063 - val_loss: 0.6914 - val_accuracy: 0.6650 - 844ms/epoch - 34ms/step
Epoch 2/30
25/25 - 0s - loss: 0.6897 - accuracy: 0.5938 - val_loss: 0.6896 - val_accuracy: 0.5700 - 54ms/epoch - 2ms/step
Epoch 3/30
25/25 - 0s - loss: 0.6857 - accuracy: 0.6075 - val_loss: 0.6861 - val_accuracy: 0.5650 - 64ms/epoch - 3ms/step
Epoch 4/30
25/25 - 0s - loss: 0.6781 - accuracy: 0.6237 - val_loss: 0.6796 - val_accuracy: 0.5650 - 60ms/epoch - 2ms/step
Epoch 5/30
25/25 - 0s - loss: 0.6664 - accuracy: 0.6500 - val_loss: 0.6670 - val_accuracy: 0.6150 - 70ms/epoch - 3ms/step
Epoch 6/30
25/25 - 0s - loss: 0.6480 - accuracy: 0.7125 - val_loss: 0.6554 - val_accuracy: 0.6100 - 53ms/epoch - 2ms/step
Epoch 7/30
25/25 - 0s - loss: 0.6238 - accuracy: 0.7225 - val_loss: 0.6336 - val_accuracy: 0.6650 - 55ms/epoch - 2ms/step
Epoch 8/30
25/25 - 0s - loss: 0.5923 - accuracy: 0.7875 - val_loss: 0.6079 - val_accuracy: 0.7400 - 63ms/epoch - 3ms/step
Epoch 9/30
25/25 - 0s 